In [1]:
import pickle
import pandas as pd
import numpy as np
import keras

2024-12-20 09:01:04.734659: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-20 09:01:04.748251: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1734685264.762919   14826 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1734685264.767393   14826 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-20 09:01:04.782296: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
predict_target = "price"
numerical_features = ['size', 'width', 'height', 'depth', 'discount_amount']

In [3]:
model = keras.saving.load_model("artifacts/model.keras")

test_data = pd.read_parquet('artifacts/X_test.parquet')

with open('artifacts/feature_scaler.pkl', 'rb') as handle:
    feature_scaler = pickle.load(handle)
with open('artifacts/label_scaler.pkl', 'rb') as handle:
    label_scaler = pickle.load(handle)

2024-12-20 09:01:06.542171: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [4]:
# Evaluate performance on the test set
test_features = test_data.drop(predict_target, axis=1)
test_labels = test_data[[predict_target]]

print("\n Evaluate the new model against the test set:")
model.evaluate(x = test_features, y = test_labels)


 Evaluate the new model against the test set:
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0081 - mean_squared_error: 0.0081  


[0.007831843569874763, 0.007831843569874763]

In [5]:
# Get all test predictions
prediction = model(test_features)
unscaled_predictions = label_scaler.inverse_transform(prediction)

In [6]:
predicted_df = test_data.copy()
predicted_df[[predict_target]] = unscaled_predictions

In [7]:
predicted_df[numerical_features] = feature_scaler.inverse_transform(predicted_df[numerical_features])

In [8]:
def undummify(df, prefix_sep="_"):
    cols2collapse = {
        item.split(prefix_sep)[0]: (prefix_sep in item) for item in df.columns
    }
    series_list = []
    for col, needs_to_collapse in cols2collapse.items():
        if needs_to_collapse:
            undummified = (
                df.filter(like=col)
                .idxmax(axis=1)
                .apply(lambda x: x.split(prefix_sep, maxsplit=1)[1])
                .rename(col)
            )
            series_list.append(undummified)
        else:
            series_list.append(df[col])
    undummified_df = pd.concat(series_list, axis=1)
    return undummified_df

In [9]:
categores = [col for col in predicted_df.columns if "category" in col]

In [10]:
concat_categories = undummify(predicted_df[categores])
predicted_df = predicted_df.drop(categores, axis=1)
predicted_df["category"] = concat_categories

In [11]:
# See where we predict a price below 0
predicted_df[predicted_df["price"]<0]

,price,sellable_online,other_colors,depth,height,width,discounted,width_d,height_d,depth_d,discount_amount,size,category
1869,-216.316431,1,0,21.000000,40.000000,22.000000,0,1,1,1,0.0,18480.0,Nursery furniture
3405,-364.528536,1,1,37.150376,38.000000,60.000000,0,1,1,0,0.0,2280.0,TV & media furniture
3440,-77.066240,1,0,37.150376,99.664062,132.285714,0,0,0,0,0.0,1.0,TV & media furniture
402,-189.115427,1,0,51.000000,15.000000,50.000000,1,1,1,1,2.0,38250.0,Bookcases & shelving units
1064,-193.497150,1,0,37.493671,113.408696,98.750000,0,0,0,0,0.0,1.0,Cabinets & cupboards
705,-117.151404,1,0,1.000000,4.000000,63.000000,0,1,1,1,0.0,252.0,Bookcases & shelving units
511,-81.429401,1,0,36.922737,131.097285,8.000000,0,1,0,0,0.0,8.0,Bookcases & shelving units
3396,-41.498299,1,0,37.150376,38.000000,60.000000,0,1,1,0,0.0,2280.0,TV & media furniture
573,-36.614123,1,0,28.000000,131.097285,9.000000,0,1,0,1,0.0,252.0,Bookcases & shelving units
3379,-77.066240,1,0,37.150376,99.664062,132.285714,0,0,0,0,0.0,1.0,TV & media furniture
